# Tucker Model

In [49]:
# Libraries/packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math 

### Camp Structure

In [50]:
Nb = 8100          # Number of people in isoboxes.
mub = 10           # Isoboxes mean occupancy (people).
hb = Nb / mub      # Number of isoboxes. 
iba = 0.5          # Proportion of area covered by isoboxes.

Nt = 10600         # Number of people in tents.
mut = 4            # Tents occupancy of (people).
ht = 10600 / mut   # Number of tents.

fblocks = np.array([1,1])   # initial sectoring.
N = Nb + Nt                 # Total population.


### Emperical Age and Sex Distribution

In [51]:
path_to_file = 'age_and_sex.csv'            # Observed data.
age_and_sex = pd.read_csv(path_to_file)     # Data frame. V1 = age, V2 is sex (1 = male?, 0  = female?).
#type(age_and_sex)

# Remove "Unamed column" and preview.
age_and_sex = age_and_sex.loc[:, ~age_and_sex.columns.str.contains('^Unnamed')]
age_and_sex.head() 

,V1,V2
0,2.1,1
1,25.6,0
2,23.6,0
3,49.6,1
4,28.6,0


In [52]:
age_and_sex.describe()            # Summary stats.
age_and_sex = age_and_sex.values  # dataframe to 2D array.
#type(age_and_sex)
#age_and_sex[:5,]

### Transmission parameters

In [53]:
# Infection
twh = 0.5   # Probability of infecting each person in your household per day.
aip = 0.1   # Probability of infecting each person you meet per meeting (Fang et al.)
tr = 1      # Initial transmission reduction (relative to assumed per contact transmission rate, outside household only).

### Other parameters

In [54]:
siprob = 0        # Probability of spotting symptoms, per person per day.
clearday = 7      # Days in quarantine after no virus shedding (i.e., recovery).
pac = 0.179       # Permanently asymptomatic cases (Mizumoto et al 2020 Eurosurveillance).
ss = 0.20         # Realtive strength of interaction between different ethnicities.

### Initial movement parameters
Note that the initial assumption is that
everyone uses the larger radius some proportion of the time, which is
__NOT__ the same as assuming that some people always use the larger radius,
Nonetheless, I am setting the proportion equal to the number of males age 10-50 in the population.

In [55]:
lr1 = 0.02       # Smaller movement radius
lr2 = 0.1        # Larger movement radius
lrtol = 0.02     # Scale interactions - two people with completely overlapping rages with this radius interact once per day

### Create population matrix (pop)
Columns:
0. Home number
1. Disease state: 0 = susceptible, 1 = exposed, 2 = presymptomatic, 3 = symptomatic, 4 = mild, 5 = severe, 6 = recovered. Similar states in quarentine are the same plus seven.
2. Days to symptoms for this person
3. Days passed in current state
4. Whether this person will be asymptomatic
5. Age
6. Male
7. Chronic
8. Wanderer (Uses the larger radius).

pop is $N \times 9$

#### Columns 0-1 (House number and disease state)

- Randomly assign each person to a household (rN).
- Draw a sample from uniform distribution between 1-810 (hb). Meaning we chose a random isobox.
- Then repeat that 8100 (Nb) times. AKA assign each person to an isobox.
- Do the same for the tent group.
- Indices (ui) of the unique array that reconstruct the input array (rN).
- The number of inidces should equal total population (N).
- Start each person as susceptible (col 1 = 0).
- Randomly assign one person to be exposed to the virus (pop[someindex,1]=1).

In [56]:
rN = np.concatenate((np.ceil(hb*np.random.uniform(0,1,Nb)), hb+np.ceil(ht*np.random.uniform(0,1,Nt)))) 
# plt.hist(rN,bins='auto') # max(rN) = 3460. why?

U,ui = np.unique(rN, return_inverse=True)
# U[ui] == rN

pop = np.column_stack((np.sort(ui), np.zeros((ui.shape)))) # Size N x 2 (18700 x 2).
# pop.shape

pop[np.random.randint(0,N),1] = 1

In [57]:
pop.shape

(18700, 2)

#### Columns 2-4
- Col 2: Days to first symptoms (if they develop symptoms) for each person, following (Backer et al. 2020 Eurosurveillance)
- Col 3: Days passed in current state, 0 for now.
- Col 4: Whether this person will be asymptomatic.

In [58]:
k = (2.3/6.4)**(-1.086)
L = 6.4 / (math.gamma(1 + 1/k))
pop = np.column_stack( (pop, k*np.random.weibull(L,(N,1)), np.zeros((N,1)), np.random.uniform(0,1,N)<pac*(N/(N-300))) )

In [59]:
pop.shape

(18700, 5)

#### Columns 5-6 
- Get N random samples from the distribution in the observed data with repetition.
- Assign age and sex following the observed distribution in the camp.

In [60]:
# Is this approach correct?
pop = np.column_stack((pop,
                       np.random.choice(age_and_sex[:,0],N, replace=True),
                       np.random.choice(age_and_sex[:,1], N, replace=True)))


# I believe I treated the age and sex distributions as separate whereas I think I should treat
# the age and sex for a give individual together (if that makes sense?).
# i.e. I got the sex and age samples on their own where as the original Matlab code gets them
# from the age-sex pairs: age_and_sex(randsample(size(age_and_sex,1),N,true),:)

In [61]:
pop.shape

(18700, 7)

In [ ]:
#num_rows, num_cols = age_and_sex.shape
#num_rows
#np.random.randint(num_rows, size = N)

In [ ]:
%Assign chronic states, matching the relationship between age and chronic
%states in the data
c_adj=fminsearch(@(adj)abs(300-sum((1+exp(-(adj-11.69+.2191*pop(:,6)-0.001461*pop(:,6).^2))).^-1)),4);
rchron=(1+exp(-(c_adj-11.69+.2191*pop(:,6)-0.001461*pop(:,6).^2))).^-1;
pop=[pop,rand(N,1)<rchron];
pop(pop(:,8)==1,5)=0;       %assue that people with chronic conditions are not asymptomatic, and correct number of asymptomatics above
pop=[pop,(pop(:,7)==1 & pop(:,6)>=10)];